In [115]:
# This python notebook annotate the bioconda recipes using the ids from bio.tools in the external session 
# in bioconda recipes. Two different input should be provided, the bioconda_recipes_path in your local machine and
# the file with the mapping.   


import glob2
import os

import requests
import sys
import yaml
import getopt
import re

from ruamel.yaml import YAML

from yaml.constructor import ConstructorError
from yaml.scanner import ScannerError

In [116]:
def get_bioconda_yalm(bioconda_path):
    yalm_files = dict()
    matches =  glob2.glob(bioconda_path + '/**/*.yaml', with_matches=True)
    yaml = YAML(typ='jinja2')
    yaml.allow_duplicate_keys = True
    yaml.allow_unicode = True
    if(len(matches) > 0 ):
        for file_name in matches:
            if(os.path.isfile(file_name[0])):
                print(file_name[0])
                try:
                    with open(file_name[0]) as fp:
                         yalm_file = yaml.load(fp)
                    path_list = file_name[0].replace(bioconda_path, "").split(os.sep)
                    software_name = path_list[1]
                    print(software_name)
                    yalm_files[software_name] = yalm_file
                except (ScannerError, ConstructorError, TypeError, AttributeError) as error:
                    print("ERROR | " + file_name[0])
                    print(error)
                    #with open(file_name[0], "w") as file:
                    #    iterable = file.read()
                    #final_description = get_bioconda_yalm(iterable)
                    #path_list = file_name[0].replace(bioconda_path, "").split(os.sep)
                    #software_name = path_list[0]
                    #yalm_files[software_name] = finale_description
                

In [117]:
bioconda_recipes_path = "/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes"
yalm_files = get_bioconda_yalm(bioconda_recipes_path)

/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/2pg_cartesian/meta.yaml
2pg_cartesian
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/abawaca/meta.yaml
abawaca
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/abricate/meta.yaml
abricate
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/abruijn/meta.yaml
abruijn
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/abundancebin/meta.yaml
abundancebin
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/abyss/meta.yaml
abyss
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/adam/meta.yaml
adam
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/addrg/meta.yaml
addrg
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/aenum/meta.yaml
aenum
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/afterqc/meta.yaml
af

besst
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bfc/meta.yaml
bfc
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bgt/meta.yaml
bgt
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bio-vcf/meta.yaml
bio-vcf
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bio_assembly_refinement/meta.yaml
bio_assembly_refinement
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bio_hansel/meta.yaml
bio_hansel
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioawk/meta.yaml
bioawk
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/biobambam/meta.yaml
biobambam
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioblend/meta.yaml
bioblend
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioconductor-a4core/meta.yaml
bioconductor-a4core
/Users/yperez/IdeaProjects/github-repo/biodo

bioconductor-biovizbase
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioconductor-biseq/meta.yaml
bioconductor-biseq
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioconductor-bsgenome/meta.yaml
bioconductor-bsgenome
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioconductor-bsgenome.celegans.ucsc.ce10/meta.yaml
bioconductor-bsgenome.celegans.ucsc.ce10
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioconductor-bsgenome.celegans.ucsc.ce6/meta.yaml
bioconductor-bsgenome.celegans.ucsc.ce6
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioconductor-bsgenome.dmelanogaster.ucsc.dm3/meta.yaml
bioconductor-bsgenome.dmelanogaster.ucsc.dm3
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioconductor-bsgenome.ecoli.ncbi.20080805/meta.yaml
bioconductor-bsgenome.ecoli.ncbi.20080805
/Users/yperez/IdeaProjects/github-repo/biodocker/bioco

bioconductor-ensembldb
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioconductor-exomecopy/meta.yaml
bioconductor-exomecopy
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioconductor-fastseg/meta.yaml
bioconductor-fastseg
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioconductor-fdb.infiniummethylation.hg19/meta.yaml
bioconductor-fdb.infiniummethylation.hg19
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioconductor-flowai/meta.yaml
bioconductor-flowai
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioconductor-flowcl/meta.yaml
bioconductor-flowcl
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioconductor-flowcore/meta.yaml
bioconductor-flowcore
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioconductor-flowdensity/meta.yaml
bioconductor-flowdensity
/Users/yperez/IdeaProjects/github-repo/biod

bioconductor-limma
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioconductor-lpsymphony/meta.yaml
bioconductor-lpsymphony
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioconductor-m3drop/meta.yaml
bioconductor-m3drop
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioconductor-maanova/meta.yaml
bioconductor-maanova
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioconductor-macat/meta.yaml
bioconductor-macat
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioconductor-macorrplot/meta.yaml
bioconductor-macorrplot
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioconductor-made4/meta.yaml
bioconductor-made4
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioconductor-makecdfenv/meta.yaml
bioconductor-makecdfenv
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/bioconductor-mant

...

fwdpy
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/fwdpy11/meta.yaml
fwdpy11
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/gadem/meta.yaml
gadem
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/galaxy-lib/meta.yaml
galaxy-lib
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/galaxy-ops/meta.yaml
galaxy-ops
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/galaxy_sequence_utils/meta.yaml
galaxy_sequence_utils
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/gap2seq/meta.yaml
gap2seq
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/garnet/meta.yaml
garnet
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/gat/meta.yaml
gat
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/gatk/meta.yaml
gatk
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/

gsort
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/gtf_to_genes/meta.yaml
gtf_to_genes
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/gubbins/meta.yaml
gubbins
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/guidescan/meta.yaml
guidescan
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/gustaf/meta.yaml
gustaf
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/gvcf-regions/meta.yaml
gvcf-regions
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/gvcftools/meta.yaml
gvcftools
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/hap.py/meta.yaml
hap.py
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/hapbin/meta.yaml
hapbin
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/hapflk/meta.yaml
hapflk
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/

jq
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/json-collect-data-source/meta.yaml
json-collect-data-source
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/junit-xml/meta.yaml
junit-xml
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/kaiju/meta.yaml
kaiju
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/kallisto/meta.yaml
kallisto
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/kaptive/meta.yaml
kaptive
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/khmer/meta.yaml
khmer
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/kid/meta.yaml
kid
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/kissplice/meta.yaml
kissplice
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/kiwidist/meta.yaml
kiwidist
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/r

metaprob
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/metaseq/meta.yaml
metaseq
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/metaseq-all/meta.yaml
metaseq-all
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/metasv/meta.yaml
metasv
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/metavelvet/meta.yaml
metavelvet
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/metavelvet-annois/meta.yaml
metavelvet-annois
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/metavelvet-sl/meta.yaml
metavelvet-sl
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/metavelvet-sl-feature-extraction/meta.yaml
metavelvet-sl-feature-extraction
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/recipes/metavelvet-sl-pipeline/meta.yaml
metavelvet-sl-pipeline
/Users/yperez/IdeaProjects/github-repo/biodocker/bioconda-recipes/r

ScannerError: while scanning for the next token
found character '\t' that cannot start any token
  in "<unicode string>", line 17, column 43:
     ... hes/process-reads-fasta.py.patch	
                                         ^ (line: 17)

In [34]:
for description in yalm_files:
    print(description)

TypeError: 'NoneType' object is not iterable